# Metody Obliczeniowe w Nauce i Technice Laboratorium 6
## Page searcher
### Paweł Gorgolewski

In [36]:
import os
import numpy as np
import pickle
import csv
import wikipedia as wiki
from wikipedia.exceptions import WikipediaException
from collections import namedtuple, defaultdict
from functools import reduce

#### Generating wiki dump
Dump indeksów artykułów z wikipedii został pobrany poprzez link
https://dumps.wikimedia.org/enwiki/20220401/

Mając tytuły wszystkich artykuł, zaciągamy zawartość tytułu poprzez bilbiotekę *wikipedia*. Następnie zapisujemy pickle pythonowe w celu łatwego ładowania do programu

In [24]:
def get_first_n_titles(n=1000, file='data\wiki-pages-indexes.txt'):
    titles = []
    with open(file, "r", encoding='utf8') as f:
        csv_reader = csv.reader(f, delimiter=":")
        for row in csv_reader:
            if n < 0:
                break

            title = row[-1]
            if '/' in title:
                continue

            titles.append(title)
            n-=1

    return titles

def get_articles_content_and_save_pickle(titles):
    not_matched = 0
    for title in titles:
        try:
            wiki_page = wiki.page(title)
            wiki_content = wiki_page.content
            #now saving pickle
            pickle.dump(wiki_content, open(f'data\\{title}', "wb"))
        except WikipediaException:
            not_matched += 1

    print(f"Not matched {not_matched} titles")

titles = get_first_n_titles()
get_articles_content_and_save_pickle(titles)

ConnectionError: ('Connection aborted.', ConnectionAbortedError(10053, 'Nawiązane połączenie zostało przerwane przez oprogramowanie zainstalowane w komputerze-hoście', None, 10053, None))

## Ładowanie artykułów

In [29]:
def get_articles(to_omit_file = 'wiki-pages-indexes.txt'):
    data_by_title = dict()
    for title in pickles():
        data_by_title[title] = dict()
        if title == to_omit_file:
            continue
        data_by_title[title] = pickle.load(open(f"data\\{title}", "rb"))

    pickle.dump(data_by_title, open(f"data\\articles_dict_dump_of_len_{len(data_by_title)}", "wb"))
    return data_by_title

def pickles(path='data'):
    for content in os.listdir(path):
        if os.path.isfile(os.path.join(path, content)):
            yield content

In [30]:
articles = get_articles()

## Tworzenie zbioru wszystkich słów

In [56]:
ArticleData = namedtuple('ArticleData', ['words_data', 'words_num'])

def parse_article(content):
    def default_value():
        return 0

    words = content.split()
    words_data = defaultdict(default_value)

    for word in words:
        if word in words_data:
            continue

        words_data[word] = words.count(words)

    return ArticleData(words_data=words_data, words_num=len(words))

def parse_all_articles(articles):
    parsed_articles = dict()
    for article in articles:
        parsed_articles[article] = parse_article(articles[article])

    return parsed_articles


def create_all_words_data(articles_data):
    total_words_count = sum(article.words_num for article in articles_data.values())
    all_words = list(set().union(*list(set(article.words_data.keys()) for article in articles_data.values())))
    words_by_count = {word: sum((article.words_data[word] for article in articles_data.values())) for word in all_words}

In [57]:
parsed_articles = parse_all_articles(articles)

In [58]:
create_all_words_data(parsed_articles)